## 42_连续子数组的最大和

输入一个int型数组，数组里面有正数也有负数。截取数组中连续的一段作为一个子数组并对子数组求和。求sum最大的那个子数组。要求时间复杂度为O(n)。

例如输入数组为：[1, -2, 3, 10, -4, 7, 2, -5]，那么和最大的子数组为[3, 10, -4, 7, 2]，sum = 18。

### 分析
如果用最直观的方法解，即枚举数组的所有子树组并求它们的和，那么一个长度为n的数组，总共有 n(n+1)/2个子数组。一个一个计算出所有的子数组的和再比较，最快也需要$O(n^2)$的时间。下面讨论时间更快的方法。

[//]: # (<img src="images/img123.png" style="width: 400px;"/>)

### 解法一：举例分析数组的规律
一步一步考虑这个数组[1, -2, 3, 10, -4, 7, 2, -5]，

• sum初始化为0。第一步加上第一个数字1，sum=1；第二步加-2, sum=-1；第三步加3, sum=2。这里就能注意到，由于此前累加的和是-1，小于0，如果用前面的sum加上3，得到的和是2，比3本身还小。所以我们可以直接弃用前面的数字，直接从3开始。结论就是，**如果前面数字的和是负数**，那么就**直接弃用**前面的数字，从下一个数字开始找sub-array。


• 从3开始，sum=3；加10，sum=13；加-4，sum=9。这里开始有两种情况：
- 之后加一个正数，sum变得比13大
- sum=13有可能已经是最大的和了
因此要把之前得到的max_sum=13先保存下来。之后加7，sum=16 --> max_sum=16；加2，sum=18，--> max_sum=18；最后是-5，到了原数组末尾，于是不考虑-5。

In [4]:
def find_greatest_sum_of_subarray(array):
    if array is None or array == [] or not all(isinstance(x, int) for x in array):
        return []

    sum = 0
    max_sum = array[0] # max_sum should not be initialized as 0!
    start_index, last_index = 0, 0

    for i in range(len(array)):
        if sum <= 0:
            sum = array[i]
            start_index = i
        else:
            sum += array[i]

        if sum > max_sum:
            max_sum = sum
            last_index = i

    if start_index > last_index: # every elemet is a negative int
        return array[last_index:last_index+1], max_sum
    else:
        return array[start_index:last_index+1], max_sum

In [7]:
# Test
test1 = [1, -2, 3, 10, -4, 7, 2, -5, -2, -1]
print(find_greatest_sum_of_subarray(test1))

test2 = [-2, -8, -1, -5, -9]
print(find_greatest_sum_of_subarray(test2))

test3 = [4, 3, 7, 10, 11]
print(find_greatest_sum_of_subarray(test3))

([3, 10, -4, 7, 2], 18)
([-1], -1)
([4, 3, 7, 10, 11], 35)


### 应用动态规划法
如果用函数法f(i)表示以第i个数字结尾的子数组的最大和，那么我们需要求出max{f(i)}，其中0<=i<n。我们可以用如下递归公式求f(i)：

$$ f(i)=\left\{
\begin{array}{rcl}
& array[i]                      & {i=0 \lor f(i-1) \leq 0}\\
& f(i-1) + array[i]             & {i \ne 0 \land f(i-1) > 0}
\end{array} \right. $$

此公式的意义：当以第i-1个数字结尾的子数组中所有数字的和小于0时，如果把这个负数与第i个数累加，则得到的结果比第i个数字本身还要小，所以这种情况下以第i个数字结尾的子数组就是第i个数字本身。（也就是之前解法一中，丢掉所有之前sum<=0 的数字）。

如果以第i-1个数字结尾的子数组f(i-1)中所有数字的和大于0，则与第i个数字累加-->就得到以第i个数字结尾的子数组f(i)中所有数字的和。

其实思路和之前是异曲同工的，只不过换了递归的思想。